In [1]:
import pandas as pd
#!pip install pyreadstat
#!pip install py7zr
import py7zr
import calendar
import os

In [114]:
features_group = pd.read_excel(r'.\codes\feature_groups.xlsx')
demo_order = pd.read_excel(r'.\codes\demo_order.xlsx')
periods = pd.read_excel(r'.\codes\periods.xlsx', sheet_name ='period_lbl')
time_period_type = pd.read_excel(r'.\codes\periods.xlsx', sheet_name ='time_period_type')
year = pd.read_excel(r'.\codes\periods.xlsx', sheet_name ='year')
segments_order = pd.read_excel(r'.\codes\segments_order.xlsx')

In [3]:
example = pd.read_excel('../Example.xlsx', sheet_name = 'example')
legends = pd.read_excel('../Example.xlsx', sheet_name = 'legends')

In [4]:
archive = py7zr.SevenZipFile('../data_in/out-for-datatile.7z', 'r')  
archive.extractall(path="/tmp")
path = '/tmp/out-for-datatile'
files = os.listdir(path)
files


['br_allshops.parquet',
 'br_tot.parquet',
 'deterg_allshops.parquet',
 'deterg_tot.parquet',
 'diapers_allshops.parquet',
 'diapers_tot.parquet',
 'femcare_allshops.parquet',
 'femcare_tot.parquet',
 'haircond_allshops.parquet',
 'haircond_tot.parquet',
 'shampoo_allshops.parquet',
 'shampoo_tot.parquet']

In [84]:
cat_dict = {'br':'MALE B&R', 'deterg': 'Laundry Detergents','diapers': 'Diapers','femcare': 'Feminine Care', 
                'haircond': 'Hair Conditioners', 'shampoo':'Shampoos' }
time_period_type_dict ={ 24 :'2MATs/ 104 we', 12:'MAT/ 52 we', 3:'3MMT/ 12we' }

In [120]:
def get_df_in_v2(path, file):
    categoty  = cat_dict[file.split('_')[0]]
    type_of_file = file.split('_')[1]
    df_in = pd.read_parquet(f'{path}\\{file}')
    df_in = df_in.rename({'Category Name':'Product Name', 'Buyer Group Name':'buyers_gr_label'
                           }, axis=1 )
    df_in['Category Name'] = [categoty] * len( df_in)
    
    col_to_int = ['duration', 'year', 'month']
    for col in col_to_int:
        df_in[col] = df_in[col].astype(int)
    
    df_in['time_period_type'] = df_in['duration'].map(time_period_type_dict) 
    #Ниже нужно оптимизировать функцию, по 10 секунд выполняется
    df_in['period_lbl'] = df_in[['time_period_type', 'year', 'month']].apply(
        lambda row:f"{row['time_period_type'].split('/')[0]}: {calendar.month_abbr[row['month']]} {row['year']}", axis=1 )  
    print(df_in.shape)
    df_in = df_in.merge(features_group, on =['Category Name', 'Product Name'], how = 'left') 
    print(df_in.shape)
    df_in = df_in.merge(demo_order, on =['buyers_gr_label'], how = 'left')
    print(df_in.shape)
    df_in = df_in.merge(periods, on =['period_lbl'], how = 'left') 
    print(df_in.shape)
    df_in = df_in.merge(time_period_type, on =['time_period_type'], how = 'left')
    print(df_in.shape)
    df_in = df_in.merge(year, on =['year'], how = 'left')
    print(df_in.shape)
    df_in = df_in.merge(segments_order, on =['Segment'], how = 'left')
    print(df_in.shape)
    return df_in
    

In [121]:
df_in_one = get_df_in_v2(path, files[0])

(210575, 34)
(210575, 42)
(210575, 47)
(210575, 49)
(210575, 50)
(210575, 51)
(210575, 52)


In [122]:
for col in df_in_one:
    print(col,sum(df_in_one[col].isnull()))

Segment 0
Product Name 0
position_name_shop 0
value 0
volume_gl 0
volume_SU 0
volume_pack 0
trips_raw 0
trips 0
buyers 0
buyers_raw 0
population 0
penetration 0
frequency 0
spend_per_buyer 0
spend_per_trip 0
volume_per_buyer_g 0
volume_per_buyer_su 0
volume_per_buyer_pack 0
volume_per_trip_g 0
volume_per_trip_su 0
volume_per_trip_pack 0
avg_price_g 0
avg_price_su 0
avg_price_pack 0
avg_pack_size 0
month_end 0
duration 0
buyers_gr_label 0
year 0
month 0
Category Name 0
time_period_type 0
period_lbl 0
Vendor Product ID 0
feature 0
feature_code 0
category_code 0
product_code 0
full_label 0
product_hier 0
product_lvls 0
Buyer Group ID 0
demo_code 0
demo_hier 0
demo_lvls 0
socdem_gr 0
period_code 24249
label_num 24249
time_period_code 47251
year_code 0
segment_code 0


In [126]:
df_in_one.loc[df_in_one['period_code'].isnull() , ['year', 'month', 'duration','period_lbl' ]]

,year,month,duration,period_lbl
38028,2024,6,24,2MATs: Jun 2024
38029,2024,6,24,2MATs: Jun 2024
38030,2024,6,24,2MATs: Jun 2024
38031,2024,6,24,2MATs: Jun 2024
38032,2024,6,24,2MATs: Jun 2024
...,...,...,...,...
210570,2024,6,3,3MMT: Jun 2024
210571,2024,6,3,3MMT: Jun 2024
210572,2024,6,3,3MMT: Jun 2024
210573,2024,6,3,3MMT: Jun 2024
